In [29]:
import pandas as pd
import numpy as np

In [30]:
data=pd.read_csv("GercekZamanliTuketim-01012021-17062023.csv",encoding='ISO-8859-1')

In [31]:
data['Tarih'] = pd.to_datetime(data['Tarih'], format='%d.%m.%Y', errors='coerce')

In [32]:
data['Saat'] = pd.to_datetime(data['Saat'], format='%H:%M').dt.time

In [33]:
data['ds'] = data['Tarih'].astype(str) + ' ' + data['Saat'].astype(str)

In [34]:
data=data.iloc[:,2:]

In [35]:
data

,Tüketim Miktarý (MWh),ds
0,"29.489,46",2021-01-01 00:00:00
1,"28.067,11",2021-01-01 01:00:00
2,"26.527,08",2021-01-01 02:00:00
3,"25.327,19",2021-01-01 03:00:00
4,"24.719,72",2021-01-01 04:00:00
...,...,...
21538,"35.118,56",2023-06-17 10:00:00
21539,"35.880,67",2023-06-17 11:00:00
21540,"34.860,03",2023-06-17 12:00:00
21541,"35.102,91",2023-06-17 13:00:00


In [36]:
data=data.rename(columns={"Tüketim Miktarý (MWh)":"y"})
data['y'] = data['y'].str.replace('.', '')
data['y'] = data['y'].str.replace(',', '.')
data['y'] = data['y'].astype(float)
test=data[data.ds>="2023-06-01"]
train=data[data.ds<"2023-06-01"]

/tmp/ipykernel_23878/2053694950.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['y'] = data['y'].str.replace('.', '')


In [37]:
test.head(1)

,y,ds
21144,33377.19,2023-06-01 00:00:00


In [38]:
train.head(1)

,y,ds
0,29489.46,2021-01-01 00:00:00


In [39]:
from prophet import Prophet

model = Prophet(seasonality_mode='multiplicative',seasonality_prior_scale=0.1)

model.add_seasonality(name='hourly', period=24, fourier_order=5)

model.fit(train)
y_test=test["y"]

forecast = model.predict(test.iloc[:,1:])



Initial log joint probability = -191.202
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       56540.4     0.0032773       1674.66      0.3444      0.3444      123   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       56589.5    0.00223027       1224.45           1           1      241   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       56736.6    0.00115928       422.468           1           1      357   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       56757.8     0.0102781       2189.54      0.1694           1      475   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       56768.5    0.00401009       1748.39           1           1      585   
    Iter      log prob        ||dx||      ||grad||       alpha  

In [40]:
error=(abs((list(y_test) - forecast["yhat"])/list(y_test))*100).mean()

In [41]:
error

3.9658216856623563

In [42]:
forecast[["ds","yhat"]]

,ds,yhat
0,2023-06-01 00:00:00,34834.426974
1,2023-06-01 01:00:00,33527.811048
2,2023-06-01 02:00:00,32698.182897
3,2023-06-01 03:00:00,32004.225107
4,2023-06-01 04:00:00,31267.457387
...,...,...
394,2023-06-17 10:00:00,38866.979386
395,2023-06-17 11:00:00,39100.361123
396,2023-06-17 12:00:00,38612.594106
397,2023-06-17 13:00:00,38164.248610


In [43]:
test

,y,ds
21144,33377.19,2023-06-01 00:00:00
21145,31906.82,2023-06-01 01:00:00
21146,30775.27,2023-06-01 02:00:00
21147,30055.67,2023-06-01 03:00:00
21148,29810.46,2023-06-01 04:00:00
...,...,...
21538,35118.56,2023-06-17 10:00:00
21539,35880.67,2023-06-17 11:00:00
21540,34860.03,2023-06-17 12:00:00
21541,35102.91,2023-06-17 13:00:00
